<a href="https://colab.research.google.com/github/dmejial/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/%7CAG3/Diego_Mejia_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AG3 -Actividad Guiada 3    
Diego Mejía    
https://github.com/dmejial/03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/%7CAG3

In [0]:
!pip install request

In [0]:
import urllib.request
file='swiss42.tsp'
urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp', file)

In [0]:
!pip install tsplib95


In [0]:
import tsplib95
import random
import math 

In [0]:
problem=tsplib95.load_problem(file)

Nodos=list(problem.get_nodes())

list(problem.get_nodes())
list(problem.get_edges())

problem.wfunc(0,1)

15

In [0]:
def distancia(a,b,problem):
  return problem.wfunc(a,b)

def crear_solucion(Nodos):
  solucion=[0]
  for _ in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion

def distancia_total(solucion,problem):
  distancia_ret=0
  for i in range(len(solucion)-1):
    distancia_ret+=distancia(solucion[i],solucion[i+1],problem)
  return distancia_ret+distancia(solucion[len(solucion)-1],solucion[0],problem)

 
solucion_prueba=crear_solucion(Nodos)

print(distancia_total(solucion_prueba,problem),solucion_prueba)

Busqueda Aleatoria

In [0]:
def busqueda_aleatoria(problem,N):
  Nodos=list(problem.get_nodes())
  
  mejor_solucion=[]
  mejor_distancia=10e10
  
  for i in range(N):
    solucion=crear_solucion(Nodos)
    distancia_solucion=distancia_total(solucion,problem)
    
    if distancia_solucion <mejor_distancia:
      mejor_solucion=solucion
      mejor_distancia=distancia_solucion
      
  #print(mejor_distancia,mejor_solucion)    
  #return mejor_solucion
      
  print("La mejor solución encontrada es ", end="")
  print(solucion)
  print("con una distancia total de ",end="")
  print(mejor_distancia)
  return mejor_solucion


busqueda_aleatoria(problem,100)

Busqueda Local

In [0]:
def generar_vecina(solucion,problem):
  mejor_solucion=[]
  mejor_distancia=10e10
  for i in range(1,len(solucion)-1):
    for j in range(i+1,len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina=distancia_total(vecina,problem)
      if distancia_vecina < mejor_distancia:
        mejor_solucion=vecina
        mejor_distancia=distancia_vecina
  return mejor_solucion

solucion_prueba=crear_solucion(Nodos)
generar_vecina(solucion_prueba,problem)

In [0]:
def busqueda_local(problem,N):
  mejor_solucion=[]
  mejor_distancia=10e10
  
  Nodos=list(problem.get_nodes())
  
  solucion_referencia=crear_solucion(Nodos)
  
  for i in range(N):
    vecina=generar_vecina(solucion_referencia,problem)
    distancia_vecina=distancia_total(vecina,problem)
    if distancia_vecina < mejor_distancia:
        mejor_solucion=vecina
        mejor_distancia=distancia_vecina
        
    solucion_referencia=mejor_solucion
  print(distancia_total(mejor_solucion,problem),mejor_solucion)
  return mejor_solucion
    
busqueda_local(problem,100)

Recocido Simulado

In [0]:
def probabilidad(T,d):
  r=random.random()
  
  if r <math.exp(-1*d/T)  :
    return True
  else:
    return False
  
def bajar_temperatura(T):
  return T*0.999

In [0]:
def genera_vecina_aleatorio(solucion):
  i,j=sorted(random.sample(range(1,len(solucion)),2))
  return solucion[:i]+[solucion[j]]+solucion[i+1:j]+ [solucion[i]]+solucion[j+1:]

In [0]:
def recocido_simulado(problem,TEMPERATURA):
  solucion_referencia=crear_solucion(Nodos)
  
  distancia_referencia=distancia_total(solucion_referencia,problem)
  
  mejor_solucion=[]
  mejor_distancia=10e10
  
  while TEMPERATURA > .01:
    vecina=genera_vecina_aleatorio(solucion_referencia)
    distancia_vecina=distancia_total(vecina,problem)
    
    if distancia_vecina < mejor_distancia:
      mejor_solucion=vecina
      mejor_distancia=distancia_vecina
      
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA,abs(distancia_referencia-distancia_vecina)):
      solucion_referencia=vecina
      distancia_referencia=distancia_vecina
      
    TEMPERATURA=bajar_temperatura(TEMPERATURA)
    
    print(mejor_distancia,mejor_solucion)
    
    return mejor_solucion
    
recocido_simulado(problem,1000)
      
      

Colonia de Hormigas

In [0]:
def Add_Nodo(problem,H,T):
  Nodos=list(problem.get_nodes())
  return random.choice( list(set(range(1,len(Nodos)))-set(H)))

def Incrementa_Feromona(problem,T,H):
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H,problem)
  return T

def Evaporar_Feromonas(T):
  T=[[max(T[i][j]-0,3,.1)  for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T


In [0]:
def hormigas(problem,N):
  Nodos=list(problem.get_nodes())
  Aristas=list(problem.get_edges())
  
  T=[[1 for _ in range(len(Nodos))] for _ in range(len(Nodos))]
  Hormiga=[[0] for _ in range(N)]
  
     
  for h in range(N):
    for i in range(len(Nodos)-1):
      Nuevo_Nodo=Add_Nodo(problem,Hormiga[h],T)
      T=Incrementa_Feromona(problem,T,Hormiga[h])
      T=Evaporar_Feromonas(T)
  mejor_solucion=[]
  mejor_distancia=10e10
  for h in range(N):
    distancia_actual=distancia_total(Hormiga[h],problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion=Hormiga[h]
      mejor_distancia=distancia_actual
      
  return mejor_solucion
 
hormigas(problem,1000)